<a href="https://colab.research.google.com/github/kcalizadeh/phil_nlp/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Mounting Drive

In [1]:
# this cell mounts drive, sets the correct directory, then imports all functions
# and relevant libraries via the functions.py file
from google.colab import drive
import sys

# install relevent libraries not included with colab
!pip install lime
!pip install symspellpy

drive.mount('/gdrive',force_remount=True)

drive_path = '/gdrive/MyDrive/Colab_Projects/Phil_NLP'

sys.path.append(drive_path)

Mounted at /gdrive


In [2]:
from functions import *
%load_ext autoreload
%autoreload 2

np.random_seed=17

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load the Data

In [3]:
df = pd.read_csv('/gdrive/MyDrive/Colab_Projects/Phil_NLP/phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,sentence_length,sentence_lowered,lemmatized_str,tokenized_txt
5112,complete works,Plato,plato,"But when he has also got an account of it, he ...","But when he has also got an account of it, he ...",102,"but when he has also got an account of it, he ...",but when -PRON- have also get an account of -...,"['But', 'when', 'he', 'has', 'also', 'got', 'a..."
287030,elements of right,Hegel,german_idealism,"In Plato's republic, subjective freedom is not...","In Plato's republic, subjective freedom is not...",154,"in plato's republic, subjective freedom is not...","in Plato 's republic , subjective freedom be ...","['In', 'Plato', ""'s"", 'republic', ',', 'subjec..."
159907,quintessence,Quine,analytic,This becomes clear as soon as Vl a rephrased i...,This becomes clear as soon as Vl a rephrased i...,91,this becomes clear as soon as vl a rephrased i...,this become clear as soon as Vl a rephrase in...,"['This', 'becomes', 'clear', 'as', 'soon', 'as..."
4416,complete works,Plato,plato,When his companions become lyric on the subjec...,When his companions become lyric on the subjec...,502,when his companions become lyric on the subjec...,when -PRON- companion become lyric on the sub...,"['When', 'his', 'companions', 'become', 'lyric..."
28437,complete works,Plato,plato,So if I to tell the story of how it really cam...,So if I to tell the story of how it really cam...,170,so if i to tell the story of how it really cam...,so if -PRON- to tell the story of how -PRON- ...,"['So', 'if', 'I', 'to', 'tell', 'the', 'story'..."


In [4]:
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True,
                                                        max_len=100))

In [5]:
# check how it worked
print(df.iloc[216282]['sentence_str'])
df['gensim_tokenized'][216282]

The bumble bee is a part of the reproductive system of the clover.


['the',
 'bumble',
 'bee',
 'is',
 'part',
 'of',
 'the',
 'reproductive',
 'system',
 'of',
 'the',
 'clover']

Well with that beautiful little quote, we are ready to start training our w2v model! At first we'll focus on a single school, since a single school is more likely to have consistency in their use of a word.

Unfortunately, we didn't have much luck with just training on the texts alone. The code for it is left here for posterity, but it was when we worked with GloVe as the base that we had results that were actually useful.

### Word 2 Vec Training

#### German Idealism as a Test Case

We start by examining the texts of German Idealism to get a feel for what kind of parameters would work best.

In [38]:
def make_w2v(series, stopwords=[], size=200, window=5, min_count=5, workers=-1, 
             epochs=20, lowercase=True, sg=0, seed=17, cbow_mean=1, alpha=0.025,
             sample=0.001, use_bigrams=True, threshold=10, bigram_min=5):
  # turn the series into a list, lower it, clean it
    sentences = [sentence for sentence in series]
    if lowercase:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word.lower() for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)
    else:
      cleaned = []
      for sentence in sentences:
        cleaned_sentence = [word for word in sentence]
        cleaned_sentence = [word for word in sentence if word not in stopwords]
        cleaned.append(cleaned_sentence)

  # incorporate bigrams
    if use_bigrams:
      bigram = Phrases(cleaned, min_count=bigram_min, threshold=threshold, delimiter=b' ')
      bigram_phraser = Phraser(bigram)
      tokens_list = []
      for sent in cleaned:
        tokens_ = bigram_phraser[sent]
        tokens_list.append(tokens_)
      cleaned = tokens_list
    else:
      cleaned = cleaned

  # build the model
    model = Word2Vec(cleaned, size=size, window=window, 
                     min_count=min_count, workers=workers, seed=seed, sg=sg,
                     cbow_mean=cbow_mean, alpha=alpha, sample=sample)
    model.train(series, total_examples=model.corpus_count, epochs=epochs)
    model_wv = model.wv
    
  # clear it to avoid unwanted transference
    del model

    return model_wv

In [39]:
gi_wv = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'], threshold=12)

We can check this model by trying out a few words. For that purpose we have a testing function that tries some common word combinations.

In [54]:
pairs_to_try = [(['law', 'moral'], []),
                (['self', 'consciousness'], []),
                (['dialectic'], []),
                (['logic'], []),
]

In [40]:
test_w2v_pos_neg(gi_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- how (0.23285)
- inside (0.22697)
- original source (0.22589)
- acid (0.21729)
- df (0.21704)

Positive - ['self', 'consciousness']	Negative - []
- discovered (0.26501)
- gans (0.24422)
- upper hand (0.23032)
- blue (0.22977)
- experi (0.22654)

Positive - ['dialectic']	Negative - []
- accorded (0.27038)
- introduced (0.2446)
- ance (0.23565)
- quadratic (0.23303)
- sublating (0.23201)

Positive - ['logic']	Negative - []
- interpenetration (0.27604)
- negating (0.25452)
- mind (0.24488)
- quired (0.24322)
- defining (0.2347)



Although some of these make a modicum of sense a lot of them seem like just gibberish. Let's try messing with some parameters.



##### Trying Skip-gram instead of C-bow

In [42]:
# make a base model with the preset parameters
skip_gi_wv = make_w2v(series = df[df['school'] == 'german_idealism']['gensim_tokenized'], 
                         stopwords=[], sg=1, seed=0)

In [43]:
test_w2v(skip_gi_wv, pairs_to_try)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Positive - ['law', 'moral']	Negative - []
- philosophical works (0.26632)
- mathematical (0.25058)
- sameness (0.24794)
- ens rationis (0.24681)
- suggests (0.23576)

Positive - ['self', 'consciousness']	Negative - []
- calls (0.25984)
- elegance (0.24366)
- acquisition (0.23788)
- sine (0.232)
- final purpose (0.23005)

Positive - ['dialectic']	Negative - []
- transcendental doctrine (0.27465)
- middle terms (0.26202)
- understood (0.25658)
- applied logic (0.25506)
- clings (0.22976)

Positive - ['logic']	Negative - []
- deciding (0.30243)
- rec (0.28958)
- possible experience (0.25629)
- calculating (0.25121)
- mathematical (0.23836)



These seem mildy more sensible. Let's tweak the other parameters.

##### Parameter Testing

In [44]:
model_v1 = make_w2v(df[df['school'] == 'german_idealism']['gensim_tokenized'],
                       stopwords=[],
                       size=500,
                       window=5,
                       min_count=25,
                       epochs=10,
                       sg=1, 
                       seed=45)

len(model_v1.vocab)

2933

In [45]:
test_w2v(model_v1, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- consistent (0.16991)
- unless (0.14104)
- distinguish (0.13626)
- rose (0.13344)
- she (0.1314)

Positive - ['self', 'consciousness']	Negative - []
- performed (0.15893)
- itselfness (0.15161)
- impossibility (0.14681)
- their (0.14224)
- more closely (0.13561)

Positive - ['dialectic']	Negative - []
- empty space (0.17288)
- manifested (0.15537)
- remark (0.15018)
- associated with (0.14597)
- distance (0.13942)

Positive - ['logic']	Negative - []
- premise (0.15924)
- test (0.15442)
- cognitions (0.1468)
- slightest (0.13728)
- notion (0.13671)



Despite tweaking parameters far and wide, it's difficult to get any results that are compellingly sensible. In most cases there are one or two terms in the similarity list that make some sense but others that are just strange or unconnected

#### Trying Another School

In [46]:
cm_w2v = make_w2v(df[df['school'] == 'communism']['gensim_tokenized'],
                       stopwords=[],
                       size=700,
                       window=10,
                       min_count=10,
                       epochs=25,
                       sg=1, 
                       seed=10)

type(cm_w2v)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [47]:
pairs_to_try=[(['material', 'conditions'], []),
              (['worker'], ['owner']),
              (['alienation', 'labor'], []),
              (['capital'], [])]

In [48]:
test_w2v(cm_w2v, pairs_to_try)

Positive - ['material', 'conditions']	Negative - []
- machines (0.13411)
- at night (0.12783)
- renewal (0.12365)
- paupers (0.12364)
- current (0.11691)

Positive - ['worker']	Negative - ['owner']
- taken place (0.12344)
- peasantry (0.12229)
- public (0.12178)
- socialists (0.11708)
- governments (0.1158)

Positive - ['alienation', 'labor']	Negative - []
- subsistence (0.137)
- occupy (0.12043)
- people (0.11918)
- pole (0.11914)
- determine (0.1153)

Positive - ['capital']	Negative - []
- actively (0.15668)
- consisting (0.12746)
- component (0.12398)
- issued (0.11602)
- growth (0.11515)



Here the results were similar - a few words that made some sense and plenty that were just odd.  

### Transfer Learning with GloVe

We'll import GloVe vectors as w2v, then use those as a base from which to train new vectors that are tuned to our corpus.

In [6]:
glove_file = datapath('/gdrive/MyDrive/Colab_Projects/Phil_NLP/glove.6B.50d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [9]:
# check out how GloVe works on our test pairs
test_w2v_pos_neg(glove_vectors, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- morality (0.82654)
- legal (0.82652)
- laws (0.81529)
- constitutional (0.80616)
- fundamental (0.80217)

Positive - ['self', 'consciousness']	Negative - []
- sense (0.83446)
- mind (0.79755)
- vision (0.78202)
- belief (0.78031)
- life (0.77984)

Positive - ['dialectic']	Negative - []
- hegelian (0.88376)
- dialectical (0.83417)
- dialectics (0.80672)
- materialist (0.77674)
- metaphysics (0.77488)

Positive - ['logic']	Negative - []
- reasoning (0.81405)
- intuitionistic (0.76531)
- concepts (0.75831)
- logical (0.75604)
- theory (0.75026)



Ok these make a lot more sense right from the start. But we want them to be trained on our actual philosophical texts - that way we can see how different thinkers use different words and potentially use the vectors for classification.

So in the cells below we train the existing GloVe model on on the German Idealist texts as a test.

In [56]:
# isolate the relevant school
documents = df[df['school'] == 'german_idealism']['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=20, threshold=10, delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=20, threshold=10, delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
base_model = Word2Vec(size=300, min_count=5)
base_model.build_vocab(trigramed_tokens)
total_examples = base_model.corpus_count

# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
base_model.train(trigramed_tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv
del base_model

In [57]:
test_w2v(base_model_wv, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- moral law (0.80435)
- freedom (0.79501)
- rule (0.79358)
- morality (0.78824)
- highest good (0.78263)

Positive - ['self', 'consciousness']	Negative - []
- self consciousness (0.9147)
- externality (0.87614)
- immediacy (0.8751)
- essence (0.87467)
- negativity (0.86413)

Positive - ['dialectic']	Negative - []
- doctrine (0.91102)
- antinomy (0.90098)
- exposition (0.89884)
- method (0.8963)
- deduction (0.89234)

Positive - ['logic']	Negative - []
- pure reason (0.85686)
- doctrine (0.84364)
- method (0.83159)
- idealism (0.81921)
- metaphysics (0.81001)



We can immediately see that these make a lot more sense (and the similarity scores are a lot higher). Self-consciousness is commonly associated with freedom in German idealism, logic with metaphysics, and the moral law with universality and the good. This is a massive improvement - these vectors can be fairly said to reflect how german idealists use these terms. Moreover, they are significantly different than the original GloVe model, which indicates that there was real learning going on here.

For comparison, let's check these same terms, but as used by Phenomenologists.

In [52]:
def train_glove(school, glove_vectors, threshold=10, stopwords=[],
                min_count=20):
  # isolate the relevant school
  documents = df[df['school'] ==school]['gensim_tokenized']

  # format the series to be used
  stopwords = []

  sentences = [sentence for sentence in documents]
  cleaned = []
  for sentence in sentences:
    cleaned_sentence = [word.lower() for word in sentence]
    cleaned_sentence = [word for word in sentence if word not in stopwords]
    cleaned.append(cleaned_sentence)

  # get bigrams
  bigram = Phrases(cleaned, min_count=min_count, threshold=threshold, 
                   delimiter=b' ')
  bigram_phraser = Phraser(bigram)

  bigramed_tokens = []
  for sent in cleaned:
      tokens = bigram_phraser[sent]
      bigramed_tokens.append(tokens)

  # run again to get trigrams
  trigram = Phrases(bigramed_tokens, min_count=min_count, threshold=threshold, 
                    delimiter=b' ')
  trigram_phraser = Phraser(trigram)

  trigramed_tokens = []
  for sent in bigramed_tokens:
      tokens = trigram_phraser[sent]
      trigramed_tokens.append(tokens)

  # build a toy model to update with
  model = Word2Vec(size=300, min_count=5)
  model.build_vocab(trigramed_tokens)
  total_examples = model.corpus_count

  # add GloVe's vocabulary & weights
  model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

  # train on our data
  model.train(trigramed_tokens, total_examples=total_examples, epochs=model.epochs)
  model_wv = model.wv
  del model
  return model_wv

In [55]:
ph_model = train_glove(school='phenomenology', glove_vectors=glove_vectors)

test_w2v(ph_model, pairs_to_try)

Positive - ['law', 'moral']	Negative - []
- linguistic (0.99409)
- categorial (0.99366)
- defining (0.99345)
- chain (0.99338)
- clarifying (0.9928)

Positive - ['self', 'consciousness']	Negative - []
- existence (0.95972)
- nature (0.95558)
- potentiality (0.95472)
- authentic (0.94812)
- understanding (0.94194)

Positive - ['dialectic']	Negative - []
- formerly (0.99032)
- confusion (0.99006)
- consideration (0.98976)
- dialogue (0.98914)
- reorientation (0.98834)

Positive - ['logic']	Negative - []
- transformation (0.98935)
- definition (0.98631)
- explication (0.9863)
- ity (0.98405)
- articulation (0.98383)



These are substantially different - which is good since the schools are substantially different. They mostly make sense, except for words like 'dialectic' which are rarely used by phenomenologists. The general attitude towards logic as a cold sterilizing force is evident, as is their emphasis on perception and natural life, as compared to the German idealist emphasis on abstractions like universality and freedom (see 'self consciousness'). 

These vectors seem to be an effective tool for revealing word usage between the schools. 

As a final kind of exploration of this method, we'll train w2v models in this way for each school and examine how each of them looks a couple of the same words. 

In [58]:
w2v_dict = {}

for school in df['school'].unique():
  w2v_dict[school] = train_glove(school, glove_vectors=glove_vectors)
  print(f'{school} completed')

plato completed
aristotle completed
empiricism completed
rationalism completed
analytic completed
continental completed
phenomenology completed
german_idealism completed
communism completed
capitalism completed


In [59]:
for school in df['school'].unique():
  print(f'\t{school.upper()}')
  print('----------------------')
  test_w2v(w2v_dict[school], [(['philosophy'], [])])

	PLATO
----------------------
Positive - ['philosophy']	Negative - []
- relief (0.95119)
- parentage (0.94101)
- flesh (0.94014)
- caution (0.9373)
- youth (0.93688)

	ARISTOTLE
----------------------
Positive - ['philosophy']	Negative - []
- cretan (0.86985)
- student (0.86937)
- practice (0.86075)
- studied (0.85763)
- economy (0.85579)

	EMPIRICISM
----------------------
Positive - ['philosophy']	Negative - []
- practice (0.95716)
- religion (0.95474)
- history (0.94577)
- inquiry (0.94528)
- vulgar (0.93813)

	RATIONALISM
----------------------
Positive - ['philosophy']	Negative - []
- church (0.95102)
- passage (0.94926)
- beginning (0.94339)
- verses (0.93945)
- treatise (0.93924)

	ANALYTIC
----------------------
Positive - ['philosophy']	Negative - []
- semantics (0.88169)
- philosophical (0.86495)
- scope (0.85655)
- science (0.85403)
- inductive logic (0.84086)

	CONTINENTAL
----------------------
Positive - ['philosophy']	Negative - []
- history (0.96353)
- speech (0.94236)


Interestingly, many of these top words align quite strongly with the school's general attitude towards philosophy. 

The model seems solid - our next step is to train one on the entire corpus for use in classification. We do that, and export it, below.

In [63]:
documents = df['gensim_tokenized']

# format the series to be used
stopwords = []

sentences = [sentence for sentence in documents]
cleaned = []
for sentence in sentences:
  cleaned_sentence = [word.lower() for word in sentence]
  cleaned_sentence = [word for word in sentence if word not in stopwords]
  cleaned.append(cleaned_sentence)

# get bigrams
bigram = Phrases(cleaned, min_count=30, threshold=10, 
                  delimiter=b' ')
bigram_phraser = Phraser(bigram)

bigramed_tokens = []
for sent in cleaned:
    tokens = bigram_phraser[sent]
    bigramed_tokens.append(tokens)

# run again to get trigrams
trigram = Phrases(bigramed_tokens, min_count=30, threshold=10, 
                  delimiter=b' ')
trigram_phraser = Phraser(trigram)

trigramed_tokens = []
for sent in bigramed_tokens:
    tokens = trigram_phraser[sent]
    trigramed_tokens.append(tokens)

# build a toy model to update with
all_text_model = Word2Vec(size=300, min_count=5)
all_text_model.build_vocab(trigramed_tokens)
total_examples = all_text_model.corpus_count

# add GloVe's vocabulary & weights
all_text_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on our data
all_text_model.train(trigramed_tokens, total_examples=total_examples, 
                     epochs=all_text_model.epochs)
all_text_wv = all_text_model.wv


As a test case, let's see how the philosophy thinks of itself as compared to how glove thinks of philosophy.

In [65]:
for model in [1, 2]:
  if model == 1:
    print(f'\tPHILOSOPHY CORPUS')
    print('------------------------------------')
    test_w2v(all_text_wv, [(['philosophy'], [])])
  if model == 2:
    print(f'\tBASE GLOVE')
    print('------------------------------------')
    test_w2v(glove_vectors, [(['philosophy'], [])])


	PHILOSOPHY CORPUS
------------------------------------
Positive - ['philosophy']	Negative - []
- metaphysics (0.79279)
- theology (0.77741)
- science (0.7549)
- philosophical (0.73011)
- psychology (0.72825)

	BASE GLOVE
------------------------------------
Positive - ['philosophy']	Negative - []
- theology (0.88151)
- philosophical (0.84362)
- mathematics (0.83389)
- psychology (0.82387)
- sociology (0.81085)



This sort of stands to reason - 'metaphysics' often has a different meaning outside of philosophical discussion, so it's not surprising to see it as the most changed term here. 

All in all, things look good, so let's export the vectors so that they can be used in our neural networks. 

In [ ]:
all_text_wv.save_word2vec_format('/gdrive/MyDrive/Colab_Projects/Phil_NLP/w2v_for_nn.bin')

And that's it! See our other notebooks for more of the modeling work. 